In [1]:
import sys
sys.path.append('../src/mane/prototype/')
import numpy as np
import graph as g
import pickle as p

from sklearn.preprocessing import normalize
from sklearn.metrics import f1_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC

In [2]:
# Load weight of experiment B3003 - Check embeddings folder for detail
with open('../src/mane/prototype/embeddings/BC3003.weights', 'rb') as f:
    w = p.load(f)
# Get embedding weight and normalize
e_norm = normalize(w[0])
# Get average embedding and nce embedding
e_nce_norm = normalize((w[0] + w[1]) / 2)

In [3]:
# Load graph and get predictor
bc = g.graph_from_pickle('../src/mane/data/blogcatalog3.graph', 
                         '../src/mane/data/blogcatalog3.community')
xids, y_train = bc.gen_community(0.5)
X_emb = [e_norm[i] for i in xids]
X_nce = [e_nce_norm[i] for i in xids]
predictor = OneVsRestClassifier(LinearSVC(random_state=0)).fit(X_emb, y_train)
predictor_nce = OneVsRestClassifier(LinearSVC(random_state=0)).fit(X_nce, y_train)

In [4]:
# Prediction using trained linear one vs rest model
y_true = [bc._communities[i] for i in bc.nodes()]
y_pred = [predictor.predict(e_norm[i].reshape(1,-1))[0] for i in bc.nodes()]
y_pred_nce =  [predictor_nce.predict(e_nce_norm[i].reshape(1,-1))[0] for i in bc.nodes()]

In [5]:
f1_score(y_true, y_pred, average='macro')

0.1611202254668174

In [6]:
f1_score(y_true, y_pred_nce, average='macro')

0.1905337605207281